# Coding Exercise: Model Agnostic Meta Learning

For this tutorial, we will be showcasing use of Model Agnostic Meta Learning, to learn a simple curve of sinusoidal data. In second part of this tutorial, which can be found on github, we will learn about how to train MAML on Mini-Image Net.

Let's Begin!!!!

#### Step 1: Import Libraries 
Let's begin by importing some simple libraries

In [8]:
import math
import random
import torch
from torch import nn
from torch.nn import functional as F
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

#### Step 2: Create a simple neural network architecture, which is going to learn the sinusoidal curve. 
As mentioned above, we will be getting randomly generated data of sinusoidal curve so, we will be using this very small network, as we don't need a big one to learn a curve.

In [9]:
def net(x, params):
    x = F.linear(x, params[0], params[1])
    x = F.relu(x)

    x = F.linear(x, params[2], params[3])
    x = F.relu(x)

    x = F.linear(x, params[4], params[5])
    return x

params = [
    torch.Tensor(32, 1).uniform_(-1., 1.).requires_grad_(),
    torch.Tensor(32).zero_().requires_grad_(),

    torch.Tensor(32, 32).uniform_(-1./math.sqrt(32), 1./math.sqrt(32)).requires_grad_(),
    torch.Tensor(32).zero_().requires_grad_(),

    torch.Tensor(1, 32).uniform_(-1./math.sqrt(32), 1./math.sqrt(32)).requires_grad_(),
    torch.Tensor(1).zero_().requires_grad_(),
]

#### step 3: set up parameters for training 
so as we are going to use inner loop vs outer loop training as mentioned in MAML paper. thus, we need to set certain parameters such as alpha, beta, learning rate, optimizer, and number of loops.

In [10]:
opt = torch.optim.SGD(params, lr=1e-2)
n_inner_loop = 5
alpha = 3e-2

#### Step 4:  Implement Optimization Algorithm

In [11]:
for it in range(100000): # training for 1 million iterations
    b = 0 if random.choice([True, False]) else math.pi # setting up beta variable randomly
    
    #### Randomly obtain task 1 sinusoidal data ####
    x = torch.rand(4, 1)*4*math.pi - 2*math.pi
    y = torch.sin(x + b)
    
    #### Randomly obtain the task 2 sinusoidal data ####
    v_x = torch.rand(4, 1)*4*math.pi - 2*math.pi
    v_y = torch.sin(v_x + b)

    opt.zero_grad() # setup optimizer

    new_params = params # initialize weights for inner loop
    for k in range(n_inner_loop):
        f = net(x, new_params) # re-initialize task 2 neural network with new parameters
        loss = F.l1_loss(f, y) # set loss as L1 Loss

        # create_graph=True because computing grads here is part of the forward pass.
        # We want to differentiate through the SGD update steps and get higher order
        # derivatives in the backward pass.
        grads = torch.autograd.grad(loss, new_params, create_graph=True)
        new_params = [(new_params[i] - alpha*grads[i]) for i in range(len(params))] # update weights of inner loop

        if it % 100 == 0: 
            print ('Iteration %d -- Inner loop %d -- Loss: %.4f' % (it, k, loss))

    v_f = net(v_x, new_params) # re-initialize task 1 neural network with new parameters
    loss2 = F.l1_loss(v_f, v_y) # calculate Loss
    loss2.backward() # Backward Pass

    opt.step()

    if it % 100 == 0: 
        print ('Iteration %d -- Outer Loss: %.4f' % (it, loss2))



Iteration 0 -- Inner loop 0 -- Loss: 0.3558
Iteration 0 -- Inner loop 1 -- Loss: 0.3815
Iteration 0 -- Inner loop 2 -- Loss: 0.3788
Iteration 0 -- Inner loop 3 -- Loss: 0.3265
Iteration 0 -- Inner loop 4 -- Loss: 0.4066
Iteration 0 -- Outer Loss: 0.7631
Iteration 100 -- Inner loop 0 -- Loss: 0.9611
Iteration 100 -- Inner loop 1 -- Loss: 0.9364
Iteration 100 -- Inner loop 2 -- Loss: 0.9122
Iteration 100 -- Inner loop 3 -- Loss: 0.8883
Iteration 100 -- Inner loop 4 -- Loss: 0.8641
Iteration 100 -- Outer Loss: 1.0115
Iteration 200 -- Inner loop 0 -- Loss: 0.7295
Iteration 200 -- Inner loop 1 -- Loss: 0.7064
Iteration 200 -- Inner loop 2 -- Loss: 0.6832
Iteration 200 -- Inner loop 3 -- Loss: 0.6597
Iteration 200 -- Inner loop 4 -- Loss: 0.6356
Iteration 200 -- Outer Loss: 0.8210
Iteration 300 -- Inner loop 0 -- Loss: 0.6044
Iteration 300 -- Inner loop 1 -- Loss: 0.4955
Iteration 300 -- Inner loop 2 -- Loss: 0.3906
Iteration 300 -- Inner loop 3 -- Loss: 0.3694
Iteration 300 -- Inner loop 4 

Iteration 3100 -- Inner loop 0 -- Loss: 0.8219
Iteration 3100 -- Inner loop 1 -- Loss: 0.8056
Iteration 3100 -- Inner loop 2 -- Loss: 0.7892
Iteration 3100 -- Inner loop 3 -- Loss: 0.7721
Iteration 3100 -- Inner loop 4 -- Loss: 0.7544
Iteration 3100 -- Outer Loss: 0.8017
Iteration 3200 -- Inner loop 0 -- Loss: 0.5580
Iteration 3200 -- Inner loop 1 -- Loss: 0.5427
Iteration 3200 -- Inner loop 2 -- Loss: 0.5283
Iteration 3200 -- Inner loop 3 -- Loss: 0.5142
Iteration 3200 -- Inner loop 4 -- Loss: 0.5003
Iteration 3200 -- Outer Loss: 0.7128
Iteration 3300 -- Inner loop 0 -- Loss: 0.5483
Iteration 3300 -- Inner loop 1 -- Loss: 0.5472
Iteration 3300 -- Inner loop 2 -- Loss: 0.5462
Iteration 3300 -- Inner loop 3 -- Loss: 0.5452
Iteration 3300 -- Inner loop 4 -- Loss: 0.5441
Iteration 3300 -- Outer Loss: 0.3654
Iteration 3400 -- Inner loop 0 -- Loss: 0.5334
Iteration 3400 -- Inner loop 1 -- Loss: 0.4981
Iteration 3400 -- Inner loop 2 -- Loss: 0.4949
Iteration 3400 -- Inner loop 3 -- Loss: 0.4

Iteration 6200 -- Inner loop 0 -- Loss: 0.6972
Iteration 6200 -- Inner loop 1 -- Loss: 0.6717
Iteration 6200 -- Inner loop 2 -- Loss: 0.6460
Iteration 6200 -- Inner loop 3 -- Loss: 0.6200
Iteration 6200 -- Inner loop 4 -- Loss: 0.5935
Iteration 6200 -- Outer Loss: 0.7519
Iteration 6300 -- Inner loop 0 -- Loss: 0.4212
Iteration 6300 -- Inner loop 1 -- Loss: 0.3836
Iteration 6300 -- Inner loop 2 -- Loss: 0.3462
Iteration 6300 -- Inner loop 3 -- Loss: 0.3081
Iteration 6300 -- Inner loop 4 -- Loss: 0.2682
Iteration 6300 -- Outer Loss: 0.5457
Iteration 6400 -- Inner loop 0 -- Loss: 0.5550
Iteration 6400 -- Inner loop 1 -- Loss: 0.5392
Iteration 6400 -- Inner loop 2 -- Loss: 0.5230
Iteration 6400 -- Inner loop 3 -- Loss: 0.5064
Iteration 6400 -- Inner loop 4 -- Loss: 0.4894
Iteration 6400 -- Outer Loss: 0.3553
Iteration 6500 -- Inner loop 0 -- Loss: 0.4947
Iteration 6500 -- Inner loop 1 -- Loss: 0.3662
Iteration 6500 -- Inner loop 2 -- Loss: 0.3967
Iteration 6500 -- Inner loop 3 -- Loss: 0.4

Iteration 9300 -- Inner loop 0 -- Loss: 0.4002
Iteration 9300 -- Inner loop 1 -- Loss: 0.3419
Iteration 9300 -- Inner loop 2 -- Loss: 0.3066
Iteration 9300 -- Inner loop 3 -- Loss: 0.2700
Iteration 9300 -- Inner loop 4 -- Loss: 0.2669
Iteration 9300 -- Outer Loss: 0.7696
Iteration 9400 -- Inner loop 0 -- Loss: 0.3471
Iteration 9400 -- Inner loop 1 -- Loss: 0.3028
Iteration 9400 -- Inner loop 2 -- Loss: 0.3134
Iteration 9400 -- Inner loop 3 -- Loss: 0.3118
Iteration 9400 -- Inner loop 4 -- Loss: 0.2978
Iteration 9400 -- Outer Loss: 0.3054
Iteration 9500 -- Inner loop 0 -- Loss: 0.5700
Iteration 9500 -- Inner loop 1 -- Loss: 0.5377
Iteration 9500 -- Inner loop 2 -- Loss: 0.5049
Iteration 9500 -- Inner loop 3 -- Loss: 0.4712
Iteration 9500 -- Inner loop 4 -- Loss: 0.4360
Iteration 9500 -- Outer Loss: 0.2796
Iteration 9600 -- Inner loop 0 -- Loss: 0.7284
Iteration 9600 -- Inner loop 1 -- Loss: 0.7197
Iteration 9600 -- Inner loop 2 -- Loss: 0.7111
Iteration 9600 -- Inner loop 3 -- Loss: 0.7

Iteration 12300 -- Inner loop 0 -- Loss: 0.8017
Iteration 12300 -- Inner loop 1 -- Loss: 0.7817
Iteration 12300 -- Inner loop 2 -- Loss: 0.7617
Iteration 12300 -- Inner loop 3 -- Loss: 0.7416
Iteration 12300 -- Inner loop 4 -- Loss: 0.7214
Iteration 12300 -- Outer Loss: 0.7274
Iteration 12400 -- Inner loop 0 -- Loss: 0.4758
Iteration 12400 -- Inner loop 1 -- Loss: 0.4174
Iteration 12400 -- Inner loop 2 -- Loss: 0.3592
Iteration 12400 -- Inner loop 3 -- Loss: 0.3000
Iteration 12400 -- Inner loop 4 -- Loss: 0.2493
Iteration 12400 -- Outer Loss: 0.4526
Iteration 12500 -- Inner loop 0 -- Loss: 0.7143
Iteration 12500 -- Inner loop 1 -- Loss: 0.6655
Iteration 12500 -- Inner loop 2 -- Loss: 0.6162
Iteration 12500 -- Inner loop 3 -- Loss: 0.5659
Iteration 12500 -- Inner loop 4 -- Loss: 0.5139
Iteration 12500 -- Outer Loss: 0.4345
Iteration 12600 -- Inner loop 0 -- Loss: 0.5968
Iteration 12600 -- Inner loop 1 -- Loss: 0.5931
Iteration 12600 -- Inner loop 2 -- Loss: 0.5909
Iteration 12600 -- Inn

...

Iteration 87300 -- Inner loop 0 -- Loss: 1.0230
Iteration 87300 -- Inner loop 1 -- Loss: 0.3834
Iteration 87300 -- Inner loop 2 -- Loss: 0.1233
Iteration 87300 -- Inner loop 3 -- Loss: 0.0587
Iteration 87300 -- Inner loop 4 -- Loss: 0.0819
Iteration 87300 -- Outer Loss: 0.0629
Iteration 87400 -- Inner loop 0 -- Loss: 0.4011
Iteration 87400 -- Inner loop 1 -- Loss: 0.1659
Iteration 87400 -- Inner loop 2 -- Loss: 0.0336
Iteration 87400 -- Inner loop 3 -- Loss: 0.0744
Iteration 87400 -- Inner loop 4 -- Loss: 0.0298
Iteration 87400 -- Outer Loss: 0.1498
Iteration 87500 -- Inner loop 0 -- Loss: 1.0355
Iteration 87500 -- Inner loop 1 -- Loss: 0.5279
Iteration 87500 -- Inner loop 2 -- Loss: 0.2129
Iteration 87500 -- Inner loop 3 -- Loss: 0.1676
Iteration 87500 -- Inner loop 4 -- Loss: 0.1020
Iteration 87500 -- Outer Loss: 0.0820
Iteration 87600 -- Inner loop 0 -- Loss: 0.2664
Iteration 87600 -- Inner loop 1 -- Loss: 0.1656
Iteration 87600 -- Inner loop 2 -- Loss: 0.1545
Iteration 87600 -- Inn

Iteration 90300 -- Inner loop 0 -- Loss: 0.4943
Iteration 90300 -- Inner loop 1 -- Loss: 0.1272
Iteration 90300 -- Inner loop 2 -- Loss: 0.0119
Iteration 90300 -- Inner loop 3 -- Loss: 0.1180
Iteration 90300 -- Inner loop 4 -- Loss: 0.0175
Iteration 90300 -- Outer Loss: 0.1492
Iteration 90400 -- Inner loop 0 -- Loss: 0.7816
Iteration 90400 -- Inner loop 1 -- Loss: 0.0852
Iteration 90400 -- Inner loop 2 -- Loss: 0.1648
Iteration 90400 -- Inner loop 3 -- Loss: 0.0502
Iteration 90400 -- Inner loop 4 -- Loss: 0.2191
Iteration 90400 -- Outer Loss: 0.2593
Iteration 90500 -- Inner loop 0 -- Loss: 1.2258
Iteration 90500 -- Inner loop 1 -- Loss: 0.6048
Iteration 90500 -- Inner loop 2 -- Loss: 0.0920
Iteration 90500 -- Inner loop 3 -- Loss: 0.0284
Iteration 90500 -- Inner loop 4 -- Loss: 0.0831
Iteration 90500 -- Outer Loss: 0.1039
Iteration 90600 -- Inner loop 0 -- Loss: 0.9181
Iteration 90600 -- Inner loop 1 -- Loss: 0.0529
Iteration 90600 -- Inner loop 2 -- Loss: 0.2830
Iteration 90600 -- Inn

Iteration 93300 -- Inner loop 0 -- Loss: 0.8239
Iteration 93300 -- Inner loop 1 -- Loss: 0.0399
Iteration 93300 -- Inner loop 2 -- Loss: 0.1413
Iteration 93300 -- Inner loop 3 -- Loss: 0.0474
Iteration 93300 -- Inner loop 4 -- Loss: 0.0924
Iteration 93300 -- Outer Loss: 0.0750
Iteration 93400 -- Inner loop 0 -- Loss: 0.2409
Iteration 93400 -- Inner loop 1 -- Loss: 0.1377
Iteration 93400 -- Inner loop 2 -- Loss: 0.0445
Iteration 93400 -- Inner loop 3 -- Loss: 0.0687
Iteration 93400 -- Inner loop 4 -- Loss: 0.0486
Iteration 93400 -- Outer Loss: 0.1600
Iteration 93500 -- Inner loop 0 -- Loss: 0.6519
Iteration 93500 -- Inner loop 1 -- Loss: 0.1201
Iteration 93500 -- Inner loop 2 -- Loss: 0.1086
Iteration 93500 -- Inner loop 3 -- Loss: 0.0706
Iteration 93500 -- Inner loop 4 -- Loss: 0.1201
Iteration 93500 -- Outer Loss: 0.0796
Iteration 93600 -- Inner loop 0 -- Loss: 1.2142
Iteration 93600 -- Inner loop 1 -- Loss: 1.0270
Iteration 93600 -- Inner loop 2 -- Loss: 0.4939
Iteration 93600 -- Inn

Iteration 96300 -- Inner loop 0 -- Loss: 0.8601
Iteration 96300 -- Inner loop 1 -- Loss: 0.5638
Iteration 96300 -- Inner loop 2 -- Loss: 0.1987
Iteration 96300 -- Inner loop 3 -- Loss: 0.1694
Iteration 96300 -- Inner loop 4 -- Loss: 0.0485
Iteration 96300 -- Outer Loss: 0.2397
Iteration 96400 -- Inner loop 0 -- Loss: 0.3946
Iteration 96400 -- Inner loop 1 -- Loss: 0.1698
Iteration 96400 -- Inner loop 2 -- Loss: 0.0570
Iteration 96400 -- Inner loop 3 -- Loss: 0.1628
Iteration 96400 -- Inner loop 4 -- Loss: 0.0462
Iteration 96400 -- Outer Loss: 0.2912
Iteration 96500 -- Inner loop 0 -- Loss: 0.8298
Iteration 96500 -- Inner loop 1 -- Loss: 0.1146
Iteration 96500 -- Inner loop 2 -- Loss: 0.1381
Iteration 96500 -- Inner loop 3 -- Loss: 0.1376
Iteration 96500 -- Inner loop 4 -- Loss: 0.1025
Iteration 96500 -- Outer Loss: 0.1784
Iteration 96600 -- Inner loop 0 -- Loss: 0.5015
Iteration 96600 -- Inner loop 1 -- Loss: 0.0876
Iteration 96600 -- Inner loop 2 -- Loss: 0.0877
Iteration 96600 -- Inn

Iteration 99300 -- Inner loop 0 -- Loss: 1.0745
Iteration 99300 -- Inner loop 1 -- Loss: 0.0846
Iteration 99300 -- Inner loop 2 -- Loss: 0.0419
Iteration 99300 -- Inner loop 3 -- Loss: 0.1036
Iteration 99300 -- Inner loop 4 -- Loss: 0.0423
Iteration 99300 -- Outer Loss: 0.1502
Iteration 99400 -- Inner loop 0 -- Loss: 0.2875
Iteration 99400 -- Inner loop 1 -- Loss: 0.2020
Iteration 99400 -- Inner loop 2 -- Loss: 0.0439
Iteration 99400 -- Inner loop 3 -- Loss: 0.1213
Iteration 99400 -- Inner loop 4 -- Loss: 0.0519
Iteration 99400 -- Outer Loss: 0.1866
Iteration 99500 -- Inner loop 0 -- Loss: 0.8138
Iteration 99500 -- Inner loop 1 -- Loss: 0.0764
Iteration 99500 -- Inner loop 2 -- Loss: 0.0880
Iteration 99500 -- Inner loop 3 -- Loss: 0.0699
Iteration 99500 -- Inner loop 4 -- Loss: 0.0793
Iteration 99500 -- Outer Loss: 0.1811
Iteration 99600 -- Inner loop 0 -- Loss: 0.8821
Iteration 99600 -- Inner loop 1 -- Loss: 0.1368
Iteration 99600 -- Inner loop 2 -- Loss: 0.1211
Iteration 99600 -- Inn

#### Step 5: Plot Obtained Results
Once we have obtained right parameters, we will first generate some random data points to sub-sample 5 data points.

In [12]:
# Randomly generate 5 data points.
t_b = math.pi 
t_x = torch.rand(4, 1)*4*math.pi - 2*math.pi
t_y = torch.sin(t_x + t_b)

opt.zero_grad()

t_params = params
for k in range(n_inner_loop):
    t_f = net(t_x, t_params)
    t_loss = F.l1_loss(t_f, t_y)

    grads = torch.autograd.grad(t_loss, t_params, create_graph=True)
    t_params = [(t_params[i] - alpha*grads[i]) for i in range(len(params))]

# 
If we will plot the results, we will see that net was able to obtain correct curve on sinusoidal data points

In [13]:
test_x = torch.arange(-2*math.pi, 2*math.pi, step=0.01).unsqueeze(1)
test_y = torch.sin(test_x + t_b)

test_f = net(test_x, t_params)

plt.plot(test_x.data.numpy(), test_y.data.numpy(), label='sin(x)')
plt.plot(test_x.data.numpy(), test_f.data.numpy(), label='net(x)')
plt.plot(t_x.data.numpy(), t_y.data.numpy(), 'o', label='Examples')
plt.legend()
plt.savefig('maml-sine.png')